In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import openmeteo_requests
import requests_cache
from retry_requests import retry

from columnas_seleccionadas import COLUMNAS_SELECCIONADAS
from tarifa_electrica import calcular_tarifa_electrica_general, cargos_por_anio, periodos

In [34]:
df = pd.read_csv('Dataset_csv/Dataset.csv')
df.columns

df.shape

(1185, 404)

In [35]:

df = df[COLUMNAS_SELECCIONADAS]
df['DIA'] = pd.to_datetime(df['DIA'], errors='coerce')
df['Anio'] = df['DIA'].dt.year
df['Mes'] = df['DIA'].dt.month
df['Dia'] = df['DIA'].dt.day

# INCLUSIÓN DÍA DE LA SEMANA

dias_semana = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miercoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sabado',
    'Sunday': 'Domingo'
}

df['Dia_semana'] = df['DIA'].dt.day_name().map(dias_semana)
print(df['Dia_semana'].unique()) 
# --------------------------

# INCLUSIÓN TEMPERATURA AMBIENTE
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# El cambio clave está en esta URL
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 32.5672,
    "longitude": -116.6251,
    "start_date": "2020-01-01",
    "end_date": "2023-12-31",
    "hourly": "temperature_2m"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
    start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
    end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
    freq=pd.Timedelta(seconds=hourly.Interval()),
    inclusive="left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data=hourly_data)


hourly_dataframe["date_local"] = hourly_dataframe["date"].dt.tz_convert("America/Mexico_City")
hourly_dataframe["DIA"] = hourly_dataframe["date_local"].dt.date

temperaturas_diarias = (
    hourly_dataframe.groupby("DIA")["temperature_2m"]
    .mean()
    .reset_index()
    .rename(columns={"temperature_2m": "Temperatura_amb"})
)

df["DIA"] = pd.to_datetime(df["DIA"]).dt.date
temperaturas_diarias["DIA"] = pd.to_datetime(temperaturas_diarias["DIA"]).dt.date

df = df.merge(temperaturas_diarias, on="DIA", how="left")


# ------------------------------

#df.drop(columns=['DIA'], inplace=True)
print(df.shape)

['Miercoles' 'Jueves' 'Viernes' 'Sabado' 'Domingo' 'Lunes' 'Martes']
(1185, 36)


In [36]:
df.head()

,DIA,EE Planta / Hl,EE Elaboracion / Hl,EE Bodega / Hl,EE Cocina / Hl,EE Agua / Hl,ET Planta / Hl,ET Elab/Hl,ET Bodega/Hl,ET Cocina/Hl,...,ET Servicios (Mj),Tot L3. L4 y Planta de CO2,Tot A40/240/50/60/Centec/Filtro,Tot A130/330/430,Tot Trasiego,Anio,Mes,Dia,Dia_semana,Temperatura_amb
0,2020-07-01,642.727209,47.145349,69.023256,0.000000,4.372093,3506.412338,924.646747,146.731163,0.000000,...,37003.371429,0.0,0.0,0.0,0.0,2020,7,1,Miercoles,18.919001
1,2020-07-02,7.767254,0.769609,0.798838,0.319229,-0.023412,67.023237,17.419777,2.050417,10.574475,...,38859.440306,0.0,0.0,0.0,0.0,2020,7,2,Jueves,21.044001
2,2020-07-03,8.801205,0.862593,0.835762,0.260924,0.126352,73.462669,20.504276,1.970632,9.604473,...,36842.992312,0.0,0.0,0.0,0.0,2020,7,3,Viernes,24.950249
3,2020-07-04,5.175639,0.439225,0.371077,0.258048,0.077983,49.022234,17.832753,1.275730,8.182167,...,47446.033620,0.0,0.0,0.0,0.0,2020,7,4,Sabado,25.531500
4,2020-07-05,7.924665,0.802365,0.717787,0.301592,0.114267,62.150576,25.156634,1.363221,10.285670,...,34880.711617,0.0,0.0,0.0,0.0,2020,7,5,Domingo,27.466917


In [37]:
df = calcular_tarifa_electrica_general(df, periodos, cargos_por_anio)

In [38]:
df.columns

Index(['DIA', 'EE Planta / Hl', 'EE Elaboracion / Hl', 'EE Bodega / Hl',
       'EE Cocina / Hl', 'EE Agua / Hl', 'ET Planta / Hl', 'ET Elab/Hl',
       'ET Bodega/Hl', 'ET Cocina/Hl', 'ET Envasado/Hl', 'Hl de Mosto',
       'Cocimientos Diarios', 'Planta (Kw)', 'Bodega (Kw)', 'Calderas (Kw)',
       'Efluentes (Kw)', 'Frio (Kw)', 'Prod Agua (Kw)', 'KW CO2',
       'KW Enfluentes Hidr', 'Kw Compresores Aire', 'Produccion (Hl)',
       'Temp Tq Intermedio', 'Gas Planta (Mj)', 'ET Envasado (Mj)',
       'ET Servicios (Mj)', 'Tot L3. L4 y Planta de CO2',
       'Tot A40/240/50/60/Centec/Filtro', 'Tot  A130/330/430', 'Tot  Trasiego',
       'Anio', 'Mes', 'Dia', 'Dia_semana', 'Temperatura_amb',
       'Tarifa_electrica'],
      dtype='object')

In [39]:
def _get_estacion_hemisferio_norte(fecha):
    mes = fecha.month
    dia = fecha.day
    
    # Invierno: 21 de Dic - 19 de Mar
    if (mes == 12 and dia >= 21) or (mes == 1) or (mes == 2) or (mes == 3 and dia < 20):
        return "Invierno"
    # Primavera: 20 de Mar - 20 de Jun
    elif (mes == 3 and dia >= 20) or (mes == 4) or (mes == 5) or (mes == 6 and dia < 21):
        return "Primavera"
    # Verano: 21 de Jun - 22 de Sep
    elif (mes == 6 and dia >= 21) or (mes == 7) or (mes == 8) or (mes == 9 and dia < 23):
        return "Verano"
    # Otoño: 23 de Sep - 20 de Dic
    elif (mes == 9 and dia >= 23) or (mes == 10) or (mes == 11) or (mes == 12 and dia < 21):
        return "Otoño"
    else:
        return "Indefinido"

def agregar_columna_estacion(df, col_anio='Anio', col_mes='Mes', col_dia='Dia'):
    
    df_resultado = df.copy()
    
    columnas_fecha = {col_anio: 'year', col_mes: 'month', col_dia: 'day'}
    try:
        df_resultado['fecha_temp'] = pd.to_datetime(
            df_resultado[[col_anio, col_mes, col_dia]].rename(columns=columnas_fecha)
        )
    except Exception as e:
        print(f"Error al convertir columnas a fecha: {e}")
        print("Asegúrate de que las columnas 'anio', 'mes' y 'dia' existan y sean correctas.")
        return df
    
    df_resultado['estacion'] = df_resultado['fecha_temp'].apply(_get_estacion_hemisferio_norte)
    
    df_resultado = df_resultado.drop(columns=['fecha_temp'])
    
    return df_resultado


df = agregar_columna_estacion(df, col_anio='Anio', col_mes='Mes', col_dia='Dia')
df.columns

Index(['DIA', 'EE Planta / Hl', 'EE Elaboracion / Hl', 'EE Bodega / Hl',
       'EE Cocina / Hl', 'EE Agua / Hl', 'ET Planta / Hl', 'ET Elab/Hl',
       'ET Bodega/Hl', 'ET Cocina/Hl', 'ET Envasado/Hl', 'Hl de Mosto',
       'Cocimientos Diarios', 'Planta (Kw)', 'Bodega (Kw)', 'Calderas (Kw)',
       'Efluentes (Kw)', 'Frio (Kw)', 'Prod Agua (Kw)', 'KW CO2',
       'KW Enfluentes Hidr', 'Kw Compresores Aire', 'Produccion (Hl)',
       'Temp Tq Intermedio', 'Gas Planta (Mj)', 'ET Envasado (Mj)',
       'ET Servicios (Mj)', 'Tot L3. L4 y Planta de CO2',
       'Tot A40/240/50/60/Centec/Filtro', 'Tot  A130/330/430', 'Tot  Trasiego',
       'Anio', 'Mes', 'Dia', 'Dia_semana', 'Temperatura_amb',
       'Tarifa_electrica', 'estacion'],
      dtype='object')

In [40]:
df.head()

,DIA,EE Planta / Hl,EE Elaboracion / Hl,EE Bodega / Hl,EE Cocina / Hl,EE Agua / Hl,ET Planta / Hl,ET Elab/Hl,ET Bodega/Hl,ET Cocina/Hl,...,Tot A40/240/50/60/Centec/Filtro,Tot A130/330/430,Tot Trasiego,Anio,Mes,Dia,Dia_semana,Temperatura_amb,Tarifa_electrica,estacion
0,2020-07-01,642.727209,47.145349,69.023256,0.000000,4.372093,3506.412338,924.646747,146.731163,0.000000,...,0.0,0.0,0.0,2020,7,1,Miercoles,18.919001,3.637996e+07,Verano
1,2020-07-02,7.767254,0.769609,0.798838,0.319229,-0.023412,67.023237,17.419777,2.050417,10.574475,...,0.0,0.0,0.0,2020,7,2,Jueves,21.044001,6.583015e+07,Verano
2,2020-07-03,8.801205,0.862593,0.835762,0.260924,0.126352,73.462669,20.504276,1.970632,9.604473,...,0.0,0.0,0.0,2020,7,3,Viernes,24.950249,7.763415e+07,Verano
3,2020-07-04,5.175639,0.439225,0.371077,0.258048,0.077983,49.022234,17.832753,1.275730,8.182167,...,0.0,0.0,0.0,2020,7,4,Sabado,25.531500,7.513082e+07,Verano
4,2020-07-05,7.924665,0.802365,0.717787,0.301592,0.114267,62.150576,25.156634,1.363221,10.285670,...,0.0,0.0,0.0,2020,7,5,Domingo,27.466917,8.286324e+07,Verano


In [41]:
df["Frio (Kw)"] = df["Frio (Kw)"].shift(-1)


In [42]:
import os

# --- 3. Guardar el archivo ---
nombre_carpeta = "Dataset_FE"
nombre_archivo = "datos_procesados.csv"
ruta_completa = os.path.join(nombre_carpeta, nombre_archivo)

# Crear carpeta si no existe
os.makedirs(nombre_carpeta, exist_ok=True)

try:
    # Guardar todas las filas menos la última
    df.iloc[:-1].to_csv(ruta_completa, index=False, encoding='utf-8-sig')
    print(f"\n✅ ¡Éxito! DataFrame guardado en: {ruta_completa}")
except Exception as e:
    print(f"\n❌ Error al guardar el archivo: {e}")



✅ ¡Éxito! DataFrame guardado en: Dataset_FE\datos_procesados.csv
